## Aim of this script : To create a single excel file containing EEG information related to all the participants in pre-meal and post-meal scenarios

----------------------------------------------

Importing the required libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter

Listing out the participants whose data is present in the EEG data folder

In [9]:
Participant_list = ["H1", "H2", "H3", "H4", "H5", "H6"]
df_pre_list = []
df_post_list = []

Since the data in EEG data folder is in a tabular format, we can't directly feed it into our ML model,<br>
We need to flatten it out so that we can pass the EEG data as features <br>
The ModifyExcelSheet() function takes the participant's ID (H1,H2,etc.) and state (pre/post) as input<br>
Reads the EEG file related to the participant in from the "EEG Data" folder and modifies it to convert all the tabular data into a single row<br>
The final file is saved in the "EEG Data Modified" folder

In [10]:
def ModifyExcelSheet(i,state) :

    wb = load_workbook(filename="EEG Data/" + i + "_EC_" + state + "FrequencysPowerandPeak_Combined.xlsx")
    ws = wb.active
    ws.move_range("A1:I66", rows=1, cols=0, translate=True)

    index = 1

    for iter in range(1,67): 

        ws[get_column_letter(index) + str(1)] = "Channel_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Delta_value_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Delta_frequency_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Theta_value_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Theta_frequency_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Alpha_value_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Alpha_frequency_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Beta_value_" + str(iter)
        index += 1
        ws[get_column_letter(index) + str(1)] = "Beta_frequency_" + str(iter)
        index += 1

    ws[get_column_letter(index)+str(1)] = "Output"
    
    if(state=="pre"):

        ws[get_column_letter(index) + str(2)] = 0

    if(state=="post"):

        ws[get_column_letter(index) + str(2)] = 1

    for iter in range(2,68):

        ws.move_range("A" + str(iter) + ":I" + str(iter), rows=-1*(iter-2), cols=9*(iter-2), translate=True)

    wb.save(filename="EEG Data Modified/" + i + "_EC_" + state + "FrequencysPowerandPeak_Flattened.xlsx")

We loop through the participants list and convert all the data from tabular to row format (flattening the data) using the ModifyExcelSheet() function. We now read the modified EEG data and make two dataframes for pre-meal and post-meal cases

In [11]:
for i in Participant_list :

    ModifyExcelSheet(i,"pre")
    ModifyExcelSheet(i,"post")

    df_pre = pd.read_excel("EEG Data Modified/" + i + "_EC_" + "preFrequencysPowerandPeak_Flattened.xlsx")
    df_post = pd.read_excel("EEG Data Modified/" + i + "_EC_" + "postFrequencysPowerandPeak_Flattened.xlsx")

    df_pre_list.append(df_pre)
    df_post_list.append(df_post)

Pre_list_merged_df = pd.concat(df_pre_list, ignore_index=True, sort=False)
Post_list_merged_df = pd.concat(df_post_list, ignore_index=True, sort=False)

We now make a final dataframe by merging the pre-meal and post-meal case dataframes and save it as "Final_df.xlsx"

In [12]:
channel_list = []

for i in range(1,67):
    channel_list.append("Channel_" + str(i))

Pre_list_merged_df = Pre_list_merged_df.drop(channel_list, axis = 1)
Post_list_merged_df = Post_list_merged_df.drop(channel_list, axis = 1)

Final_df = pd.concat([Pre_list_merged_df,Post_list_merged_df], ignore_index=True, sort=False)
Final_df.to_excel("Final_df.xlsx")